# Ensemble Anomaly Detection

**Мета:** Об'єднати результати всіх методів в єдиний risk score.

**Методи (5 рівнів):**
1. Rule-based (44 rules) — процедурні порушення
2. Statistical (Benford, Z-score) — числові аномалії
3. Isolation Forest — глобальні outliers
4. HDBSCAN — кластеризація + outlier detection
5. Network Analysis — мережевий аналіз (картелі, монополії)

**Ensemble підхід:**
- Weighted voting: кожен метод голосує з вагою
- Consensus: скільки методів flagged тендер
- Final risk score: нормалізована комбінація

**Pipeline:**
1. Rule-based ✓
2. Statistical ✓
3. Isolation Forest ✓
4. HDBSCAN ✓
5. Network ✓
6. **Ensemble** ← current

In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.data_loader import load_tenders, load_bids, load_buyers, load_suppliers, load_bidders
from src.detectors import (
    RuleBasedDetector,
    StatisticalDetector,
    PyODDetector,
    HDBSCANDetector,
    NetworkAnalysisDetector,
    EnsembleDetector,
)

# ============================================================
# CONFIGURATION
# ============================================================
YEARS = [2022, 2023, 2024, 2025]
CONTAMINATION = 0.05
HDBSCAN_SAMPLE_SIZE = 1_000_000  # HDBSCAN on sample
# ============================================================

# Create output directories
Path('../results/figures/ensemble').mkdir(parents=True, exist_ok=True)
Path('../results').mkdir(parents=True, exist_ok=True)

# Style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print(f"Configuration: YEARS={YEARS}, CONTAMINATION={CONTAMINATION}")

Configuration: YEARS=[2022, 2023, 2024, 2025], CONTAMINATION=0.05


## 1. Завантаження даних

In [2]:
print("Loading data...")
tenders = load_tenders(years=YEARS)
bids = load_bids(years=YEARS)
buyers = load_buyers()
suppliers = load_suppliers()
bidders = load_bidders()

print(f"\nDataset:")
print(f"  Tenders: {len(tenders):,}")
print(f"  Bids: {len(bids):,}")
print(f"  Buyers: {len(buyers):,}")
print(f"  Suppliers: {len(suppliers):,}")
print(f"  Bidders: {len(bidders):,}")

Loading data...
Scanning 2022...
Scanning 2023...
Scanning 2024...
Scanning 2025...
Loaded 12,877,960 records
Scanning bids 2022...
Scanning bids 2023...
Scanning bids 2024...
Scanning bids 2025...
Loaded 2,639,473 bids
Loaded buyers: 35,995
Loaded suppliers: 358,376
Loaded bidders: 72,291

Dataset:
  Tenders: 12,877,960
  Bids: 2,639,473
  Buyers: 35,995
  Suppliers: 358,376
  Bidders: 72,291


## 2. Run All Detectors

In [ ]:
# 1. Rule-based
print("="*60)
print("1. RULE-BASED DETECTOR")
print("="*60)
rule_detector = RuleBasedDetector()
rule_results = rule_detector.detect(tenders, buyers_df=buyers, bids_df=bids)
print(f"\nRule-based anomalies (score >= 6): {(rule_results['rule_risk_score'] >= 6).sum():,}")

1. RULE-BASED DETECTOR
Processing 12,877,960 tenders...
Step 1/4: Computing aggregations...
  Computing CPV stats...
  Computing buyer stats...
  Computing supplier stats...
  Computing pair stats...
  Aggregations complete.
Step 2/4: Merging reference data...
Step 3/4: Applying 44 rules...


In [ ]:
# 2. Statistical
print("="*60)
print("2. STATISTICAL DETECTOR")
print("="*60)
stat_detector = StatisticalDetector()
stat_results = stat_detector.detect(tenders, bids_df=bids)
print(f"\nStatistical anomalies (score >= 3): {(stat_results['stat_score'] >= 3).sum():,}")

In [ ]:
# 3. Isolation Forest (via PyODDetector)
print("="*60)
print("3. ISOLATION FOREST")
print("="*60)

if_detector = PyODDetector(
    algorithm="iforest",
    contamination=CONTAMINATION,
    random_state=42,
)
if_results = if_detector.fit_detect(tenders, buyers_df=buyers, suppliers_df=suppliers)

# Rename columns for ensemble compatibility
if_results = if_results.rename(columns={"score": "if_score", "anomaly": "if_anomaly"})

print(f"IF anomalies: {if_results['if_anomaly'].sum():,}")


In [ ]:
# 4. HDBSCAN (on sample)
print("="*60)
print("4. HDBSCAN (sample)")
print("="*60)

hdbscan_detector = HDBSCANDetector(
    min_cluster_size=50,
    min_samples=10,
    contamination=CONTAMINATION,
)
hdbscan_results = hdbscan_detector.fit_detect(
    tenders, 
    buyers_df=buyers, 
    suppliers_df=suppliers,
    sample_size=HDBSCAN_SAMPLE_SIZE,
)
print(f"HDBSCAN anomalies: {hdbscan_results['hdbscan_anomaly'].sum():,}")


In [ ]:
# 5. Network Analysis
print("="*60)
print("5. NETWORK ANALYSIS")
print("="*60)

network_detector = NetworkAnalysisDetector(
    min_co_bids=3,
    min_contracts=3,
    min_community_size=3,
    # Stricter anomaly thresholds to reduce false positives
    suspicious_min_degree=10,        # Was 5
    suspicious_min_clustering=0.7,   # Was 0.5
    rotation_min_ratio=0.7,          # Was 0.5
    rotation_min_interactions=5,     # New: at least 5 head-to-head competitions
    monopoly_min_ratio=0.9,          # Was 0.8
    monopoly_min_contracts=20,       # Was 10
)
network_results = network_detector.fit_detect(tenders, bids_df=bids, bidders_df=bidders)
print(f"\nNetwork anomalies: {network_results['network_anomaly'].sum():,}")

In [ ]:
# Combine all results using EnsembleDetector
print("="*60)
print("COMBINING RESULTS WITH ENSEMBLE DETECTOR")
print("="*60)

ensemble_detector = EnsembleDetector(
    weights={
        'rule': 1.0,
        'stat': 0.8,
        'if': 1.0,
        'hdbscan': 0.8,
        'network': 1.0,
    },
    consensus_threshold=2,
)

ensemble = ensemble_detector.combine(
    rule_results=rule_results,
    stat_results=stat_results,
    if_results=if_results,
    hdbscan_results=hdbscan_results,
    network_results=network_results,
)

print(f"\nEnsemble dataset: {len(ensemble):,} tenders")

# Skip - EnsembleDetector handles normalization internally

In [ ]:
# Scores are already normalized by EnsembleDetector
print("Score columns in ensemble results:")
print([col for col in ensemble.columns if 'score' in col])

## 5. Compute Ensemble Score

In [ ]:
# EnsembleDetector computes scores and consensus automatically
# ensemble_score, consensus_count, consensus_pct, ensemble_anomaly, ensemble_risk_level

print("WEIGHTS:", ensemble_detector.weights)
print(f"\nEnsemble score stats:")
print(ensemble['ensemble_score'].describe())

In [ ]:
# Risk levels are computed by EnsembleDetector
# Based on consensus_count: critical (>=75%), high (>=50%), medium (>=25%), low (<25%)

risk_dist = ensemble['ensemble_risk_level'].value_counts()
print("\nENSEMBLE RISK DISTRIBUTION:")
for level in ['critical', 'high', 'medium', 'low']:
    count = risk_dist.get(level, 0)
    pct = count / len(ensemble) * 100
    print(f"  {level:10} {count:>10,} ({pct:>5.2f}%)")

## 6. Consensus Analysis

In [ ]:
# Consensus breakdown (5 methods)
n_methods = len(ensemble_detector.methods_used)
print(f"CONSENSUS BREAKDOWN ({n_methods} methods: {ensemble_detector.methods_used}):")

consensus_dist = ensemble['consensus_count'].value_counts().sort_index()
for count, num in consensus_dist.items():
    pct = num / len(ensemble) * 100
    methods = f"{count}/{n_methods} methods"
    print(f"  {methods}: {num:>10,} ({pct:>5.2f}%)")

# Critical: most methods agree
critical_threshold = int(n_methods * 0.75)  # 75%+ methods
critical_consensus = ensemble[ensemble['consensus_count'] >= critical_threshold]
print(f"\nCRITICAL ({critical_threshold}+ of {n_methods} methods): {len(critical_consensus):,} tenders")

In [ ]:
# Visualize consensus
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Consensus distribution
n_methods = len(ensemble_detector.methods_used)
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, n_methods + 1))
axes[0].bar(consensus_dist.index, consensus_dist.values, color=colors[:len(consensus_dist)])
axes[0].set_xlabel('Number of Methods Flagged')
axes[0].set_ylabel('Number of Tenders')
axes[0].set_title(f'Consensus Distribution ({n_methods} methods)')
axes[0].set_xticks(range(n_methods + 1))

# Risk level pie
risk_colors = {'critical': '#d62728', 'high': '#ff7f0e', 'medium': '#ffbb78', 'low': '#2ca02c'}
risk_order = ['critical', 'high', 'medium', 'low']
risk_values = [risk_dist.get(r, 0) for r in risk_order]
axes[1].pie(risk_values, labels=risk_order, colors=[risk_colors[r] for r in risk_order],
            autopct='%1.1f%%', startangle=90)
axes[1].set_title('Ensemble Risk Distribution')

plt.tight_layout()
plt.savefig('../results/figures/ensemble/consensus_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Method Contribution Analysis

In [ ]:
# Method contribution analysis using EnsembleDetector
print("METHOD STATISTICS:")
print(ensemble_detector.method_summary().to_string(index=False))

# Correlation matrix
print("\nSCORE CORRELATION MATRIX:")
corr_matrix = ensemble_detector.correlation_matrix()
print(corr_matrix.round(3).to_string())

# Agreement matrix (Jaccard similarity)
print("\nANOMALY AGREEMENT MATRIX (Jaccard):")
agreement_matrix = ensemble_detector.agreement_matrix()
print(agreement_matrix.round(3).to_string())

In [ ]:
# Visualize agreement matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Correlation heatmap
ax = axes[0]
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, ax=ax)
ax.set_title('Score Correlation Matrix')

# Agreement heatmap (Jaccard)
ax = axes[1]
sns.heatmap(agreement_matrix.astype(float), annot=True, fmt='.2f', cmap='YlOrRd', ax=ax)
ax.set_title('Anomaly Agreement Matrix (Jaccard)')

plt.tight_layout()
plt.savefig('../results/figures/ensemble/method_agreement.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Critical Tenders Analysis

In [ ]:
# Get critical tenders (using EnsembleDetector helper)
critical_tenders = ensemble_detector.get_critical_tenders()
normal_tenders = ensemble[ensemble['consensus_count'] == 0]

# Merge with tender details
critical_with_details = critical_tenders.merge(
    tenders[['tender_id', 'tender_value', 'is_single_bidder', 'is_competitive']],
    on='tender_id', how='left'
)
normal_with_details = normal_tenders.merge(
    tenders[['tender_id', 'tender_value', 'is_single_bidder', 'is_competitive']],
    on='tender_id', how='left'
)

print("="*60)
print("CRITICAL TENDERS CHARACTERISTICS")
print("="*60)

print(f"\n{'Metric':<30} {'Critical':>15} {'Normal':>15}")
print("-"*60)

# Value
c_mean = critical_with_details['tender_value'].mean()
n_mean = normal_with_details['tender_value'].mean()
print(f"{'Mean tender value (M UAH)':<30} {c_mean/1e6:>15,.2f} {n_mean/1e6:>15,.2f}")

c_med = critical_with_details['tender_value'].median()
n_med = normal_with_details['tender_value'].median()
print(f"{'Median tender value (K UAH)':<30} {c_med/1e3:>15,.1f} {n_med/1e3:>15,.1f}")

# Competition
c_sb = critical_with_details['is_single_bidder'].mean()
n_sb = normal_with_details['is_single_bidder'].mean()
print(f"{'Single bidder rate (%)':<30} {c_sb*100:>15.1f} {n_sb*100:>15.1f}")

c_comp = critical_with_details['is_competitive'].mean()
n_comp = normal_with_details['is_competitive'].mean()
print(f"{'Competitive rate (%)':<30} {c_comp*100:>15.1f} {n_comp*100:>15.1f}")

In [ ]:
# Procurement method distribution
print("\nPROCUREMENT METHOD:")

# Merge with tenders to get procurement method
critical_methods = critical_tenders.merge(tenders[['tender_id', 'procurement_method']], on='tender_id')
normal_methods = normal_tenders.merge(tenders[['tender_id', 'procurement_method']], on='tender_id')

critical_method_dist = critical_methods['procurement_method'].value_counts(normalize=True) * 100
normal_method_dist = normal_methods['procurement_method'].value_counts(normalize=True) * 100

for method in critical_method_dist.index:
    c_pct = critical_method_dist.get(method, 0)
    n_pct = normal_method_dist.get(method, 0)
    ratio = c_pct / n_pct if n_pct > 0 else 0
    print(f"  {method}: {c_pct:.1f}% (vs {n_pct:.1f}% normal) - {ratio:.1f}x")

In [ ]:
# Year distribution
print("\nYEAR DISTRIBUTION:")

critical_years = critical_tenders.merge(tenders[['tender_id', 'year']], on='tender_id')
critical_year_dist = critical_years['year'].value_counts().sort_index()

for year, count in critical_year_dist.items():
    total_year = len(tenders[tenders['year'] == year])
    pct = count / total_year * 100
    print(f"  {year}: {count:,} ({pct:.2f}% of year)")

## 9. Top Risky Entities

In [ ]:
# Top buyers by critical tender count
print("TOP 10 BUYERS BY CRITICAL TENDERS:")

critical_full = critical_tenders.merge(
    tenders[['tender_id', 'buyer_id', 'supplier_id', 'tender_value']], 
    on='tender_id', how='left'
)

top_buyers_critical = critical_full.groupby('buyer_id').agg({
    'tender_id': 'count',
    'tender_value': 'sum',
    'ensemble_score': 'mean'
}).sort_values('tender_id', ascending=False).head(10)

top_buyers_critical = top_buyers_critical.reset_index().merge(
    buyers[['buyer_id', 'buyer_name', 'buyer_region']], on='buyer_id', how='left'
)

for _, row in top_buyers_critical.iterrows():
    name = str(row['buyer_name'])[:50] if pd.notna(row['buyer_name']) else 'N/A'
    print(f"  {row['tender_id']:>5,} tenders | {row['tender_value']/1e6:>10,.1f}M UAH | {name}")

In [ ]:
# Top suppliers by critical tender count
print("\nTOP 10 SUPPLIERS BY CRITICAL TENDERS:")

top_suppliers_critical = critical_full.groupby('supplier_id').agg({
    'tender_id': 'count',
    'tender_value': 'sum',
    'ensemble_score': 'mean'
}).sort_values('tender_id', ascending=False).head(10)

top_suppliers_critical = top_suppliers_critical.reset_index().merge(
    suppliers[['supplier_id', 'supplier_name']], on='supplier_id', how='left'
)

for _, row in top_suppliers_critical.iterrows():
    name = str(row['supplier_name'])[:50] if pd.notna(row['supplier_name']) else 'N/A'
    print(f"  {row['tender_id']:>5,} tenders | {row['tender_value']/1e6:>10,.1f}M UAH | {name}")

## 10. Save Results

In [ ]:
# Save ensemble results
print("Saving results...")

# Define output columns
score_cols = [col for col in ensemble.columns if 'score' in col]
anomaly_cols = [col for col in ensemble.columns if 'anomaly' in col]
output_cols = ['tender_id'] + score_cols + anomaly_cols + ['consensus_count', 'consensus_pct', 'ensemble_risk_level']

# Save full results
ensemble[output_cols].to_csv('../results/ensemble_results.csv', index=False)
print(f"Saved full results: results/ensemble_results.csv ({len(ensemble):,} rows)")

# Save critical only
critical_tenders[output_cols].to_csv('../results/critical_tenders.csv', index=False)
print(f"Saved critical tenders: results/critical_tenders.csv ({len(critical_tenders):,} rows)")

# Save summary
summary = ensemble_detector.summary()
summary.to_csv('../results/ensemble_summary.csv', index=False)
print(f"Saved summary: results/ensemble_summary.csv")

## 11. Summary

In [ ]:
print("="*60)
print("ENSEMBLE SUMMARY")
print("="*60)

print(f"\nDataset: {len(ensemble):,} tenders ({YEARS[0]}-{YEARS[-1]})")
print(f"\nMethods combined ({len(ensemble_detector.methods_used)}):")
for method in ensemble_detector.methods_used:
    anomaly_col = f"{method}_anomaly"
    if anomaly_col in ensemble.columns:
        count = ensemble[anomaly_col].sum()
        print(f"  {method}: {count:,} anomalies")

print(f"\nENSEMBLE RISK LEVELS:")
for level in ['critical', 'high', 'medium', 'low']:
    count = risk_dist.get(level, 0)
    pct = count / len(ensemble) * 100
    print(f"  {level:10} {count:>10,} ({pct:>5.2f}%)")

print(f"\nCRITICAL TENDERS (ensemble_risk_level == 'critical'):")
print(f"  Count: {len(critical_tenders):,}")
if len(critical_with_details) > 0:
    print(f"  Total value: {critical_with_details['tender_value'].sum()/1e9:.2f}B UAH")
    print(f"  Mean value: {critical_with_details['tender_value'].mean()/1e6:.2f}M UAH")
    print(f"  Single bidder rate: {critical_with_details['is_single_bidder'].mean()*100:.1f}%")

print("\n" + "="*60)

## Висновки

### Ensemble підхід (5 методів):
- **Rule-based** — процедурні порушення (44 rules)
- **Statistical** — числові аномалії (Benford, Z-score, HHI)
- **Isolation Forest** — глобальні outliers
- **HDBSCAN** — кластеризація + outlier detection
- **Network Analysis** — картелі, монополії, bid rotation

### Scoring:
- **Weighted score**: зважена комбінація нормалізованих scores
- **Consensus voting**: кількість методів, що flagged тендер
- **Risk levels**: critical (>=75%), high (>=50%), medium (>=25%), low (<25%)

### Ключові результати:
- **Critical** (більшість методів згодні) — найвища впевненість для аудиту
- Методи доповнюють один одного (низька кореляція)
- Різні типи аномалій покриваються різними методами

### Збережені файли:
- `results/ensemble_results.csv` — всі тендери з scores
- `results/critical_tenders.csv` — лише critical
- `results/ensemble_summary.csv` — summary statistics

### EnsembleDetector:
```python
from src.detectors import EnsembleDetector

detector = EnsembleDetector(consensus_threshold=2)
results = detector.combine(
    rule_results=...,
    stat_results=...,
    if_results=...,
    hdbscan_results=...,
    network_results=...,
)

critical = detector.get_critical_tenders()
print(detector.summary())
```